## UCEC Reccurence Notebook - Pre-processing
This program processes data from ucec_tcga_pan_can_atlas_2018 to be ready for training a machine learning algorithm to predict recurrence. 

In [99]:
import config

import pandas as pd
import numpy as np
import joblib
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [100]:
# Loading in the mRNA and clinical data:
mrna_df = pd.read_csv("ucec_tcga_pan_can_atlas_2018/data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt", sep="\t", comment="#")

clinical_df = pd.read_csv("ucec_tcga_pan_can_atlas_2018\data_clinical_patient.txt", sep="\t", comment="#", low_memory=False)
clinical_df = clinical_df.set_index('PATIENT_ID')
# There are 527 patients in the mRNA and 529 patients in the clinical data

# The first 2 columns of the mRNA data are labels (Hugo_Symbol then Entrez_Gene_Id). 
# 13 of the genes do not have Hugo_symbols, so for these I will you the Entrex_Gene_Id as the label.
missing_symbols = mrna_df['Hugo_Symbol'].isnull()
mrna_df.loc[missing_symbols, 'Hugo_Symbol'] = mrna_df.loc[missing_symbols, 'Entrez_Gene_Id'].astype(str)

# There are 7 rows that have both the same Hugo_Symbol and Entrez_Gene_Id but different values for the patients.
# I will rename these rows to have unique labels by appending -1-of-2 and -2-of-2 to the Hugo_Symbol.
# Get value counts
counts = mrna_df['Hugo_Symbol'].value_counts()

# Generate unique labels for duplicates
def label_duplicates(value, index):
    if counts[value] == 1:
        return value  # Keep unique values unchanged
    occurrence = mrna_df.groupby('Hugo_Symbol').cumcount() + 1  # Count occurrences per group
    return f"{value}-{occurrence[index]}-of-{counts[value]}"

# Apply the labeling function
mrna_df['Hugo_Symbol'] = [label_duplicates(value, idx) for idx, value in mrna_df['Hugo_Symbol'].items()]

mrna_df = mrna_df.set_index('Hugo_Symbol')
mrna_df = mrna_df.drop(columns="Entrez_Gene_Id") # removing the label column before I transpose the df
mrna_df= mrna_df.transpose() # now the patients are the index and the genes are the columns
mrna_df.index = [id[:-3] for id in mrna_df.index] # removes extranious -01 so that the patient ids match the clinical data

def drop_patients_missing_data(clinical_df, mrna_df):
    # Find patient IDs not shared between the two dataframes:
    clinical_not_in_mrna = set(clinical_df.index) - set(mrna_df.index)
    mrna_not_in_clinical = set(mrna_df.index) - set(clinical_df.index)
    # There are 2 patients ('TCGA-EY-A1GJ', 'TCGA-AP-A0LQ') in the clinical data that are not in the mRNA data.
    clinical_df = clinical_df.drop(index=clinical_not_in_mrna)
    mrna_df = mrna_df.drop(index=mrna_not_in_clinical)
    return clinical_df, mrna_df

clinical_df, mrna_df = drop_patients_missing_data(clinical_df, mrna_df)
# Now both dataframes have 527 patients

In [101]:
# Check that all column labels in mrna_df are strings
non_str_cols = [col for col in mrna_df.columns if not isinstance(col, str)]
if non_str_cols: 
    raise ValueError(f"Non-string column labels found: {non_str_cols}")

In [102]:
# # Genes from https://pmc.ncbi.nlm.nih.gov/articles/PMC7565375/ 
# # and https://pmc.ncbi.nlm.nih.gov/articles/PMC9929804/ FIXME: look more into this later
# literature_genes = set([
#     "MLH1", "MSH2", "MSH6", "PMS2", "PTEN", "POLD1", "POLE", "NTHL1", "MUTYH", "BRCA1", "GINS4", "ESR1"
# ])


# def prune_correlated_features(df, threshold=0.90, literature_genes=set()):
#     corr_matrix = df.corr().abs() # this line takes a while to run
#     np.fill_diagonal(corr_matrix.values, 0) # ignore self-correlation

#     # Build adjacency map of correlations above threshold
#     high_corr_map = {
#         gene: set(corr_matrix.index[corr_matrix.loc[gene] >= threshold])
#         for gene in corr_matrix.columns
#     }

#     genes_to_keep = set(corr_matrix.columns) # start with all genes

#     while True:
#         # Find genes that are still correlated
#         correlated_genes = {g: nbrs for g, nbrs in high_corr_map.items() if nbrs & genes_to_keep}
#         if not correlated_genes:
#             break

#         # Count connections for each gene
#         degrees = {g: len(nbrs & genes_to_keep) for g, nbrs in correlated_genes.items() if g in genes_to_keep}
#         if not degrees:
#             break

#         # Choose candidate for removal
#         worst_gene = max(degrees, key=lambda g: degrees[g])

#         # If literature gene vs. non-literature → skip removal of literature
#         if worst_gene in literature_genes:
#             # Try removing one of its correlated non-literature neighbors instead
#             neighbors = correlated_genes[worst_gene] & genes_to_keep
#             non_lit_neighbors = [n for n in neighbors if n not in literature_genes]
#             if non_lit_neighbors:
#                 worst_gene = min(non_lit_neighbors, key=lambda n: df[n].var())
#             else:
#                 # Can't drop a lit gene or its only neighbors → break
#                 break
#         else:
#             # break ties by variance (drop lower variance gene)
#             ties = [g for g, d in degrees.items() if d == degrees[worst_gene]]
#             if len(ties) > 1:
#                 worst_gene = min(ties, key=lambda g: df[g].var())

#         genes_to_keep.remove(worst_gene)

#     return df[list(genes_to_keep)]

# og_mrna_shape = mrna_df.shape
# mrna_df = prune_correlated_features(
#     mrna_df, 
#     threshold=config.CORRELATION_THRESHOLD, 
#     literature_genes=literature_genes)
# print("removed ", og_mrna_shape - mrna_df.shape[1], " features")
# joblib.dump(mrna_df, "data/pruned_mrna_df.pkl")


In [103]:
mrna_df = joblib.load("data/pruned_mrna_df.pkl")

In [104]:

def assign_label(row):
    '''given a row assigns 1 for recurrance, 0 for no recurrance, 
    and None if the patient has no recurrence information. 
    Currently uses NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT to identify recurrance.
    If NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT is NaN, uses DSF_STATUS to save the label.'''
    if row['NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT'] == 'Yes':
        return 1
    elif row['NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT'] == 'No':
        return 0
    elif pd.isna(row['NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT']):
        if row['DFS_STATUS'] == '1:Recurred/Progressed':
            return 1
        elif row['DFS_STATUS'] == '0:DiseaseFree':
            return 0
        else:
            return None

def drop_highly_uniform_columns(df, threshold=0.99):
    """
    Drops columns where more than 'threshold' proportion of non-NaN values are the same.

    Parameters:
    - df: pandas DataFrame
    - threshold: float (default 0.99), proportion threshold to drop columns

    Returns:
    - pandas DataFrame with specified columns dropped
    """
    cols_to_drop = []
    for col in df.columns:
        non_na_values = df[col].dropna()
        if not non_na_values.empty:
            top_freq = non_na_values.value_counts(normalize=True).iloc[0]
            if top_freq > threshold:
                cols_to_drop.append(col)
    return df.drop(columns=cols_to_drop)


In [105]:
# Drop rows with no recurrence label
clinical_df = clinical_df.dropna(
    subset=["DFS_STATUS", "NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT"],
    how="all"
)
mrna_df = mrna_df.loc[clinical_df.index]

labels = clinical_df.apply(assign_label, axis=1)

In [106]:
from sklearn.model_selection import train_test_split

def split_train_test(clinical_df, mrna_df, labels, test_size=0.2, random_state=1):
    """
    Splits clinical and mRNA data into train/test sets using precomputed labels.

    Parameters
    ----------
    clinical_df : pd.DataFrame
        Clinical features (indexed by patient ID).
    mrna_df : pd.DataFrame
        mRNA expression features (indexed by patient ID).
    labels : pd.Series
        Precomputed labels indexed by patient ID.
    test_size : float
        Fraction of patients to hold out for testing.
    random_state : int
        Random seed for reproducibility.

    Returns
    -------
    dict of train/test splits:
        {
            "X_clinical_train", "X_clinical_test",
            "X_mrna_train", "X_mrna_test",
            "y_train", "y_test"
        }
    """

    # Train/test split on patient IDs
    train_ids, test_ids = train_test_split(
        labels.index,
        test_size=test_size,
        stratify=labels,
        random_state=random_state
    )

    # Slice dataframes and labels
    splits = {
        "X_clinical_train": clinical_df.loc[train_ids],
        "X_clinical_test":  clinical_df.loc[test_ids],
        "X_mrna_train":     mrna_df.loc[train_ids],
        "X_mrna_test":      mrna_df.loc[test_ids],
        "y_train":          labels.loc[train_ids],
        "y_test":           labels.loc[test_ids]
    }

    return splits

splits = split_train_test(clinical_df, mrna_df, labels, test_size=0.2, random_state=1)

clinical_train = splits["X_clinical_train"]
clinical_test  = splits["X_clinical_test"]
mrna_train     = splits["X_mrna_train"]
mrna_test      = splits["X_mrna_test"]
y_train          = splits["y_train"]
y_test           = splits["y_test"]



In [107]:
# remove the column if over MAX_NULL_VALS percent null values
print("length before removing null-heavy columns:", len(clinical_train.columns))
original_column_set = set(clinical_train.columns)
clinical_train = clinical_train.dropna(axis=1, thresh=len(clinical_train) * (1 - config.MAX_NULL_VALS))
print("length after removing null-heavy columns:", len(clinical_train.columns))
print(original_column_set - set(clinical_train.columns)) # checking which columns were removed


print("length before removing null-heavy columns:", len(mrna_train.columns))
original_column_set = set(mrna_train.columns)
mrna_train = mrna_train.dropna(axis=1, thresh=len(mrna_train) * (1 - config.MAX_NULL_VALS))
print("length after removing null-heavy columns:", len(mrna_train.columns))
print(original_column_set - set(mrna_train.columns)) # checking which columns were removed

# remove columns where over 99% of the non-null values are the same
clinical_train = drop_highly_uniform_columns(clinical_train)

# remove columns where over 99% of the non-null values are the same
mrna_train = drop_highly_uniform_columns(mrna_train)


cols_to_drop = [
    "DAYS_LAST_FOLLOWUP",              # follow-up time after diagnosis (future info)
    "FORM_COMPLETION_DATE",            # administrative metadata, not predictive
    "INFORMED_CONSENT_VERIFIED",       # administrative, no biological meaning
    "NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT",  # recurrence event → direct leakage
    "PERSON_NEOPLASM_CANCER_STATUS",   # disease status at follow-up → leakage
    "IN_PANCANPATHWAYS_FREEZE",        # technical/analysis flag, not biological
    "OS_STATUS",                       # overall survival outcome → leakage
    "OS_MONTHS",                       # overall survival time → leakage
    "DSS_STATUS",                      # disease-specific survival outcome → leakage
    "DSS_MONTHS",                      # disease-specific survival time → leakage
    "DFS_STATUS",                      # disease-free survival outcome → leakage
    "DFS_MONTHS",                      # disease-free survival time → leakage
    "PFS_STATUS",                      # progression-free survival outcome → leakage
    "PFS_MONTHS",                       # progression-free survival time → leakage
    "CANCER_TYPE_ACRONYM",
    "OTHER_PATIENT_ID",
    "SEX",
    "AJCC_PATHOLOGIC_TUMOR_STAGE",
    "DAYS_TO_INITIAL_PATHOLOGIC_DIAGNOSIS",
    "HISTORY_NEOADJUVANT_TRTYN",
    "PATH_M_STAGE",
    "ICD_O_3_SITE", # removed because is the same as ICD_10
    "ICD_O_3_"
]

# remove non-informational columns
clinical_train = clinical_train.drop(columns=cols_to_drop, errors='ignore')

length before removing null-heavy columns: 37
length after removing null-heavy columns: 31
{'PRIMARY_LYMPH_NODE_PRESENTATION_ASSESSMENT', 'PATH_T_STAGE', 'PATH_M_STAGE', 'ETHNICITY', 'PATH_N_STAGE', 'AJCC_PATHOLOGIC_TUMOR_STAGE'}
length before removing null-heavy columns: 20359
length after removing null-heavy columns: 17372
{'TACR3', 'PABPC1L2A', 'OR10R2', 'OR2T5', 'CDY2B', 'AMBN', 'DDI1', 'PRR23A', 'NLRP10', 'SNORD50B', 'SPANXC', 'ZPBP', 'OLFM3', 'UCMA', 'DMRTB1', 'C21orf131', 'TNFSF18', 'OR6C3', 'PER4', 'ONECUT1', 'OR1E1', 'CST11', 'VSX2', 'TTTY18', 'CSH1', 'OR5AC2', 'GRK1', 'SNORD36C', 'PATE3', 'DUSP13', 'SNORA70C', 'SNORD116-10', 'ZCCHC5', 'APOC4', 'MAS1', 'UTF1', 'OR2G2', 'SNORA38', 'FAM19A4', 'KRTAP4-9', 'SNORD121B', 'C2orf51', 'KRTAP10-5', 'PRAMEF16', 'CLEC4M', 'C15orf54', 'PMCHL1', 'SRY', 'SPINK14', 'SNORD115-10', 'FAM71B', 'TAS2R43', 'CXorf51', 'OR10G8', 'ZFP91-CNTF', 'THEG', 'PAX7', 'OR4L1', 'TRIM48', 'SNORD114-18', 'CRHR1', 'SLC24A2', 'UBTFL1', 'USP17L2', 'ZFP42', 'OR9K2', 

In [108]:
# # Just used for looking at the data for the labels
# pair_counts = X_train.groupby(["NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT", 'DFS_STATUS', "PFS_STATUS"], dropna=False).size().reset_index(name='Count')

# # Print the pairings and the count
# print(pair_counts)

# # Removes the 32 rows where we have no recurrance label (neither NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT nor DFS_STATUS are known)
# X_train = X_train.dropna(subset=['DFS_STATUS', 'NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT'], how='all')

# # numpy array for the labels for recurrance
# labels = np.array(X_train.apply(assign_label, axis=1)) 
# print(Counter(labels))


In [109]:
categorical_columns = ["ICD_10", 
                        "PRIOR_DX", 
                        "RACE",
                        "RADIATION_THERAPY", 
                        "GENETIC_ANCESTRY_LABEL"] #FIXME: do further research on what ICD_10 and ICD_O_3_SITE are


In [110]:
X_train = clinical_train.join(mrna_train, how='inner')

# --- Numerical ---
numerical_df = X_train.select_dtypes(include=['number']).copy()

# Save medians for test-time
num_medians = numerical_df.median()
numerical_df = numerical_df.fillna(num_medians)

# --- Categorical ---
categorical_df = X_train[categorical_columns].copy()

# Save modes for test-time
cat_modes = categorical_df.mode().iloc[0]   # mode() returns a DataFrame, take first row
categorical_df = categorical_df.fillna(cat_modes)

# One-Hot Encode (drop first to avoid redundancy)
categorical_df = pd.get_dummies(categorical_df, drop_first=True, dtype=float)

X_train = pd.concat([numerical_df, categorical_df], axis=1)


In [ ]:
X_test = clinical_test.join(mrna_test, how='inner')

numerical_test = X_test.select_dtypes(include=['number']).copy()
numerical_test = numerical_test.fillna(num_medians)  # <- use train medians

categorical_test = X_test[categorical_columns].copy()
categorical_test = categorical_test.fillna(cat_modes)  # <- use train modes
categorical_test = pd.get_dummies(categorical_test, drop_first=True, dtype=float)

# Align test with training columns
categorical_test = categorical_test.reindex(columns=categorical_df.columns, fill_value=0)



# Combine back
X_test = pd.concat([numerical_test, categorical_test], axis=1)

# X_test, y_train, y_test need to be preprocessed the same way as X_train
X_test = X_test[X_train.columns]

Saving my split of training and testing data

In [112]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(396, 17393) (99, 17393) (396,) (99,)


In [113]:
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

joblib.dump(X_train, "og_preproc_data_rerun/X_train.pkl")
joblib.dump(X_test, "og_preproc_data_rerun/X_test.pkl")
joblib.dump(y_train, "og_preproc_data_rerun/y_train.pkl")
joblib.dump(y_test, "og_preproc_data_rerun/y_test.pkl")
# # joblib.dump(X.columns, config.FEATURE_NAMES)

['og_preproc_data_rerun/y_test.pkl']